In [1]:
import csv
import pandas as pd
import word2vec
import nltk
import numpy as np

In [2]:
table = pd.read_csv('Documents/Dataset/merged_Table.csv')

In [3]:
table = table.drop(columns="TO BE DELETED")
table = table.drop(columns="TO BE DELETED.1")
table = table.drop(columns="TO BE DELETED.2")
table = table.drop(columns="TO BE DELETED.3")
table = table.drop(columns="TO BE DELETED.4")

In [4]:
table.head()

,Unnamed: 0,Source of ADL,DrugLabel (ADL),DBID,pubchem_id,Source of ADL.1,stitch_id_flat,stitch_id_stereo,umls_cui_from_meddra,method,umls_name,Type_of_term,meddra_id,meddra_name,drug_name,DBID.1,ind_name,ind_id
0,0,prescription/20130626_6bd94a89-f18a-45e5-d0a4-...,1 INDICATIONS AND USAGE\n\n\n\n\n\n\n\n\n1.1 S...,DB00014,47725,prescription/20130626_6bd94a89-f18a-45e5-d0a4-...,CID100047725,CID000047725,C0014175,NLP_precondition,Endometriosis,LLT,C0014175,Endometriosis,Goserelin,DB00014,Endometriosis,C0014175
1,1,prescription/20130626_6bd94a89-f18a-45e5-d0a4-...,1 INDICATIONS AND USAGE\n\n\n\n\n\n\n\n\n1.1 S...,DB00014,47725,prescription/20130626_6bd94a89-f18a-45e5-d0a4-...,CID100047725,CID000047725,C0014175,NLP_precondition,Endometriosis,PT,C0014175,Endometriosis,Goserelin,DB00014,Endometriosis,C0014175
2,2,prescription/20120908_870d88c3-4388-454b-9360-...,INDICATIONS AND USAGE\n\n\n\n\n\n\n\n\nHemophi...,DB00035,27991,prescription/20120908_870d88c3-4388-454b-9360-...,CID100027991,CID005311065,C0019069,NLP_indication,Hemophilia A,LLT,C0019069,Factor VIII deficiency,Desmopressin,DB00035,Hemophilia A,C0019069
3,3,prescription/20120908_870d88c3-4388-454b-9360-...,INDICATIONS AND USAGE\n\n\n\n\n\n\n\n\nHemophi...,DB00035,27991,prescription/20120908_870d88c3-4388-454b-9360-...,CID100027991,CID005311065,C0019069,NLP_indication,Hemophilia A,PT,C0019069,Factor VIII deficiency,Desmopressin,DB00035,Hemophilia A,C0019069
4,4,prescription/20130131_9fd48eec-aa24-4888-b99f-...,INDICATIONS AND USAGE\n\n\n\n\n\n Hemophilia A...,DB00035,27991,prescription/20130131_9fd48eec-aa24-4888-b99f-...,CID100027991,CID005311065,C0019069,NLP_indication,Hemophilia A,LLT,C0019069,Factor VIII deficiency,Desmopressin,DB00035,Hemophilia A,C0019069


Extract the ADL column in order to generate a context dataframe for text mining.

In [5]:
context = table['DrugLabel (ADL)']

In [6]:
context.head()

0    1 INDICATIONS AND USAGE\n\n\n\n\n\n\n\n\n1.1 S...
1    1 INDICATIONS AND USAGE\n\n\n\n\n\n\n\n\n1.1 S...
2    INDICATIONS AND USAGE\n\n\n\n\n\n\n\n\nHemophi...
3    INDICATIONS AND USAGE\n\n\n\n\n\n\n\n\nHemophi...
4    INDICATIONS AND USAGE\n\n\n\n\n\n Hemophilia A...
Name: DrugLabel (ADL), dtype: object

Now that we can see that the text is correctly obtained from the context dataframe, we now transform it into a list such that it is readable

In [80]:
dfToList = context.tolist()


The code from below this point will relate to the word2vec model.

This is the part which imports the gensim package which is primarily used for the word2vec model. It also includes the logging package which is useful for reading and analysing performance

In [81]:
 
# imports needed and set up logging
import gzip
import gensim 
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
 

This is a test to see if the entry gets printed out as a sentence. We wish for the context to be printed out in a similiar way.

In [83]:
data_file="reviews_data.txt.gz"

with gzip.open ('reviews_data.txt.gz', 'rb') as f:
    for i,line in enumerate (f):
        print(line)
        break

b"Oct 12 2009 \tNice trendy hotel location not too bad.\tI stayed in this hotel for one night. As this is a fairly new place some of the taxi drivers did not know where it was and/or did not want to drive there. Once I have eventually arrived at the hotel, I was very pleasantly surprised with the decor of the lobby/ground floor area. It was very stylish and modern. I found the reception's staff geeting me with 'Aloha' a bit out of place, but I guess they are briefed to say that to keep up the coroporate image.As I have a Starwood Preferred Guest member, I was given a small gift upon-check in. It was only a couple of fridge magnets in a gift box, but nevertheless a nice gesture.My room was nice and roomy, there are tea and coffee facilities in each room and you get two complimentary bottles of water plus some toiletries by 'bliss'.The location is not great. It is at the last metro stop and you then need to take a taxi, but if you are not planning on going to see the historic sites in Be

In [84]:
#data_file2 = context.to_string()

for i, line in enumerate(dfToList):
    print(line)
    if i == 1:
        break


#data_file2 = nltk.tokenize()

1 INDICATIONS AND USAGE








1.1 Stage B2-C Prostatic Carcinoma
ZOLADEX is indicated for use in combination with flutamide for the management of locally confined Stage T2b-T4 (Stage B2-C) carcinoma of the prostate. Treatment with ZOLADEX and flutamide should start 8 weeks prior to initiating radiation therapy and continue during radiation therapy [see Dosage and Administration (2.1)and Clinical Studies (14.1)]. 





1.2 Prostatic Carcinoma
ZOLADEX is indicated in the palliative treatment of advanced carcinoma of the prostate [see Dosage and Administration (2.2) and Clinical Studies (14.2)]. 





1.3 Endometriosis
ZOLADEX is indicated for the management of endometriosis, including pain relief and reduction of endometriotic lesions for the duration of therapy. Experience with ZOLADEX for the management of endometriosis has been limited to women 18 years of age and older treated for 6 months [see Dosage and Administration (2.3) andClinical Studies (14.3)]. 





1.4 Endometrial Thinn

YES! This is shown (kinda) in the same way as the previous example.

This part will now open the list and convert each word in the list into a new, tokenized list called "documents"

In [90]:
def read_input(input_file):
    """This method reads the input file which is in gzip format"""
    
    logging.info("reading file {0}...this may take a while".format(input_file))
    
    
    for i, line in enumerate (input_file): 

        if (i%10000==0):
            logging.info ("read {0} reviews".format (i))
        # do some pre-processing and return a list of words for each review text
        yield gensim.utils.simple_preprocess (line)

# read the tokenized reviews into a list
# each review item becomes a serries of words
# so this becomes a list of lists
documents = list (read_input (dfToList))
logging.info ("Done reading data file")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

2019-03-25 13:49:29,498 : INFO : read 0 reviews
2019-03-25 13:49:33,057 : INFO : read 10000 reviews
2019-03-25 13:49:37,542 : INFO : read 20000 reviews
2019-03-25 13:49:41,769 : INFO : read 30000 reviews
2019-03-25 13:49:46,190 : INFO : read 40000 reviews
2019-03-25 13:49:48,956 : INFO : Done reading data file


In [101]:
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=20)

2019-03-25 14:02:10,718 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2019-03-25 14:02:10,721 : INFO : collecting all words and their counts
2019-03-25 14:02:10,723 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-03-25 14:02:11,044 : INFO : PROGRESS: at sentence #10000, processed 1855924 words, keeping 3592 word types
2019-03-25 14:02:11,544 : INFO : PROGRESS: at sentence #20000, processed 4886712 words, keeping 4703 word types
2019-03-25 14:02:11,982 : INFO : PROGRESS: at sentence #30000, processed 7802421 words, keeping 5667 word types
2019-03-25 14:02:12,405 : INFO : PROGRESS: at sentence #40000, processed 10643351 words, keeping 6489 word types
2019-03-25 14:02:12,648 : INFO : collected 7524 word types from a corpus of 12480701 raw words and 47631 sentences
2019-03-25 14:02:12,649 : INFO : Loading a fresh vocabulary
2019-03-25 14:02:12,665 : INFO : min_count=2 retains 7524 unique words (100% of original 7524, drop

(191217146, 249614020)

In [94]:
model?

Type:           Word2Vec
String form:    Word2Vec(vocab=7524, size=150, alpha=0.025)
File:           c:\users\i6074101\appdata\local\continuum\anaconda3\lib\site-packages\gensim\models\word2vec.py
Docstring:     
Class for training, using and evaluating neural networks described in https://code.google.com/p/word2vec/

If you're finished training a model (=no more updates, only querying)
then switch to the :mod:`gensim.models.KeyedVectors` instance in wv

The model can be stored/loaded via its :meth:`~gensim.models.word2vec.Word2Vec.save()` and
:meth:`~gensim.models.word2vec.Word2Vec.load()` methods, or stored/loaded in a format
compatible with the original word2vec implementation via `wv.save_word2vec_format()`
and `Word2VecKeyedVectors.load_word2vec_format()`.
Init docstring:
Initialize the model from an iterable of `sentences`. Each sentence is a
list of words (unicode strings) that will be used for training.

Parameters
----------
sentences : iterable of iterables
    The `sentences

In [2]:
w1 = ""
model.wv.most_similar (positive=w1)

NameError: name 'model' is not defined

In [13]:
# look up top 6 words similar to 'polite'
w1 = ["polite"]
model.wv.most_similar (positive=w1,topn=6)

[('courteous', 0.9207567572593689),
 ('friendly', 0.8291413187980652),
 ('cordial', 0.8213261365890503),
 ('professional', 0.7890110611915588),
 ('attentive', 0.7848842144012451),
 ('curteous', 0.7752357721328735)]

In [12]:
model.wv.doesnt_match(["dog","cat","france"])

2019-03-25 09:42:34,928 : INFO : precomputing L2-norms of word weight vectors


'france'

In [18]:
context?

Type:        Series
String form:
0        1 INDICATIONS AND USAGE\n\n\n\n\n\n\n\n\n1.1 S...
           1        1 INDICATIONS AND USAGE\n\n\n <...> DICATIONS AND USAGE\n\n\n\n\n\n\n\n\n1.1 M...
           Name: DrugLabel (ADL), Length: 47631, dtype: object
Length:      47631
File:        c:\users\i6074101\appdata\local\continuum\anaconda3\lib\site-packages\pandas\core\series.py
Docstring:  
One-dimensional ndarray with axis labels (including time series).

Labels need not be unique but must be a hashable type. The object
supports both integer- and label-based indexing and provides a host of
methods for performing operations involving the index. Statistical
methods from ndarray have been overridden to automatically exclude
missing data (currently represented as NaN).

Operations between Series (+, -, /, *, **) align values based on their
associated index values-- they need not be the same length. The result
index will be the sorted union of the two indexes.

Parameters
----------
data :